In [60]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn import ensemble
from sklearn.model_selection import train_test_split

#### Load Symptom Dataset

In [2]:
data = pd.read_csv('./Data/final.csv', sep = ',')

####  Load Symptom Severity 

In [3]:
severity = pd.read_csv('./Data/Symptom-severity.csv')

####  Load Exam Mapping

In [4]:
mapping = pd.read_csv('./Data/map.csv', sep = ';')

# Split data for train and test

In [5]:
symptomCount = 131
testCount = 102

[train,test] = train_test_split(data,random_state=420)
trainA = np.array(train)
trainAX = trainA[:,testCount:]
trainAY = trainA[:,:testCount]

trainX=train.iloc[:,testCount:]
trainY=train.iloc[:,:testCount]
#print(trainY[0:5])

testX=test.iloc[:,testCount:]
testY=test.iloc[:,:testCount]

In [6]:
def actualAccuracies( predictions, labels):
    thresh =(predictions.max()+ predictions.min())/2
    predictions[predictions>thresh]=1
    predictions[predictions<=thresh]=0
    
    correctByLine=[]
    for i in range(len(testY)):
        matches = predictions[i]==labels[i]
        correctByLine.append(len(matches[matches==True])/len(matches))
    return([len(np.nonzero([x for x in correctByLine if x==1])[0]),
            len(np.nonzero([x for x in correctByLine if x>0.99])[0]),
            len(np.nonzero([x for x in correctByLine if x>0.95])[0]),
            len(np.nonzero([x for x in correctByLine if x>0.90])[0]),
            len(np.nonzero([x for x in correctByLine if x!=1])[0])
           ])

# Decision Tree

In [61]:
clf = ensemble.RandomForestClassifier().fit(trainX, trainY)

In [62]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print ("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print ("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ("{}return {}".format(indent, tree_.value[node]))

    recurse(0, 1)
tree_to_code(clf,testX.columns)

AttributeError: 'RandomForestClassifier' object has no attribute 'tree_'

In [63]:
preds = clf.predict(testX)
accs =  actualAccuracies(preds, testY.to_numpy())
print(accs[3]," classified 90% correctly")
print(accs[2]," classified 95% correctly")
print(accs[1]," classified 99% correctly")
print(accs[0]," classified 100% correctly")
print(accs[4]," classified incorrectly (Not 100%)")

1170  classified 90% correctly
1170  classified 95% correctly
1170  classified 99% correctly
1170  classified 100% correctly
0  classified incorrectly (Not 100%)


In [19]:
ps = clf.get_params()
ps
clf.score(testX,testY)

1.0

In [58]:
z = np.zeros(len(testX.iloc(0)[0]))
#z[114]=1
z[97]=1
oQueAMariaTem = clf.predict([z])[0]
getTests(oQueAMariaTem)

array(['Blood Tests', 'Physical Examination'], dtype=object)

In [53]:
def getTests(oneHotResults):
    dic = np.array(testY.columns)
    return dic[oneHotResults==1]

array(['Blood Tests', 'Skin biopsy', 'Patch test'], dtype=object)

In [54]:
import sys
np.set_printoptions(threshold=sys.maxsize)

for p in preds:
    print(getTests(p))

['Blood Tests' 'MRI Scan' 'CT scan' 'Sputum test' 'Biopsy'
 'Mantoux tuberculin skin test']
['Blood Tests' 'CT scan' 'Chest X-ray' 'Sputum test'
 'Pleural fluid culture' 'Pulse oximetry']
['Blood Tests' 'CT Scan' 'Echocardiogram' 'Brain MRI Scan'
 'Carotid Ultrasound' 'Cerebral angiogram']
['Physical Examination']
['Blood Tests' 'Physical Examination' 'MRI Scan' 'X-ray']
['Blood Tests' 'Skin Prick Test (SPT)' 'Intradermal Skin Test'
 'Physical Examination']
['MRI Scan' 'CT Scan' 'Neck X-ray' 'Myelophagy' 'Electromyography (EMG)']
['Blood Tests' 'Chest X-ray' 'Spirometry' 'Exhaled nitric oxide test'
 'Peak flow meter test']
['Physical Examination' 'Anoscopy' 'Rigid proctosigmoidoscopy'
 'Colonoscopy' 'Flexible sigmoidoscopy' 'Barium X-ray']
['Blood Tests' 'Physical Examination' 'Liver Ultrasound' 'Liver Biopsy'
 'MRI Scan' 'CT Scan']
['Blood Tests' 'Skin biopsy' 'Patch test']
['Blood Tests' 'CT scan' 'Chest X-ray' 'Sputum test'
 'Pleural fluid culture' 'Pulse oximetry']
['Blood Tests' '

 'Pleural fluid culture' 'Pulse oximetry']
['TSH test' 'T4 test' 'Thyroid scan' 'Thyroid ultrasound']
['Dengue NS1 antigen test' 'Serological tests' 'IgG antibody testing'
 'Nucleic acid amplification tests (NAATs)']
['Blood Tests' 'Chest X-ray' 'Spirometry' 'Exhaled nitric oxide test'
 'Peak flow meter test']
['MRI Scan' 'Electronystagmography (ENG)' 'Videonystagmography (VNG)']
['Blood Tests' 'Urine analysis' 'Stool tests' 'Bone marrow test'
 'Tissue culture']
['Blood Tests' 'Urine analysis' 'Urine Culture' 'Susceptibility testing']
['Physical Examination' 'ELISA Test' 'Saliva Test' 'Viral Load Test'
 'Western Blot']
['Blood Tests' 'Serum bilirubin test' 'Biopsy' 'Ultrasound']
['TSH test' 'T4 test' 'Thyroid scan' 'Thyroid ultrasound']
['Blood Tests' 'Physical Examination']
['Blood Tests' 'Urine analysis' 'Urine Culture' 'Susceptibility testing']
['Blood Tests' 'Serum bilirubin test' 'Biopsy' 'Ultrasound']
['MRI Scan' 'Electronystagmography (ENG)' 'Videonystagmography (VNG)']
['Blood 